In [ ]:
############# LOAD DEPENDENCIES #############
model_name = "fold3_UNET07_bg0.05_batch16_px32" # ignores debris that stains positive, quite impressive
import sys
sys.path.append('../src/') 
import SEE_TC as ctc

import tifffile
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# read whitelist, generate file_names array
scope_ID = "501592"

In [ ]:
img_dir = "/mnt/dho-nas06/zhaolab/long_term_storage/CTC/images/tiles/FF_tiles_501592/"
files = ctc.parse_nd2_paths(img_dir, "bin_open", recursive=True)
files = [s.replace(".bin_open.tiff", "") for s in files]

In [ ]:
c_names = ['740','635','555','475','390']

In [ ]:
len(files)

In [ ]:
# read in segmented tiles, >31 == 1000 candidate tiles which is plenty
img_list = []
bin_list = []

for i in range(len(files)):
    print("-------------"+str(i)+"------------")
    
    file_i = files[i]
    
    img_path_i = file_i+".tif"
    bin_path_i = file_i+".bin_open.tiff"
    
    if os.path.isfile(img_path_i) == False | os.path.isfile(bin_path_i) == False:
        print("skipping sample ("+file_i+") no input")
        print(img_path_i)
        print("###############\n")            
        continue

    img_i = tifffile.imread(img_path_i)
    img_i = np.transpose(img_i, (1,2,0)) # channel last
    img_i = img_i.astype(np.float32) # convert for processing
    print("Image shape (raw): "+str(img_i.shape))
    bin_i = tifffile.imread(bin_path_i)
    bin_i = np.expand_dims(bin_i, -1)
    bin_i = bin_i.squeeze()
    img_list.append(img_i)
    bin_list.append(bin_i)

In [ ]:
# filter to just the standard channel count images (some lack the full channel suite)
target_shape = (2304, 2304, 6)
img_list_filtered = []
bin_list_filtered = []

for array1, array2 in zip(img_list, bin_list):
    if array1.shape == target_shape:
        img_list_filtered.append(array1)
        bin_list_filtered.append(array2)


In [ ]:

# Print the shape of each array
for i, array in enumerate(img_list_filtered):
    print(f"Array {i} shape: {array.shape}")

# Function to crop arrays to (36, 1487, 1487)
def crop_to_shape(arr, target_shape=(2304, 2304)):
    return arr[:, :target_shape[1], :target_shape[2]]

# Process the list
bin_list_filtered = [crop_to_shape(arr) if arr.shape != (2304, 2304) else arr for arr in bin_list_filtered]

# Print the shape of each array (36, 1487, 1487, 6)
for i, array in enumerate(bin_list_filtered):
    print(f"Array {i} shape: {array.shape}")

In [ ]:
img_stack = np.stack(img_list_filtered, axis=0)
bin_stack = np.stack(bin_list_filtered, axis=0)
print("stack shapes: ",img_stack.shape,bin_stack.shape)

img_fg, img_bg = ctc.split_foreground_background(img_stack,bin_stack)

In [ ]:
bin_stack.shape

In [ ]:
########## BUILD FF PROFILES HERE...

In [ ]:
##### use Kask formula
c = 4 # each channel requires a bit of supervision, as outliers in the FG profile need to be cleaned if not caught by automated detection
ff_mask_BG_i,_,ff_mask_BG_i2 = ctc.build_FF_kask(img_bg[:,:,:,c], ds_dim=12, us_dim=img_fg.shape[1])

plt.imshow(ff_mask_BG_i)

In [ ]:
import numpy as np
n_splits = 3  # Number of splits
input_FG = img_fg[:,:,:,c].squeeze()
ff_mask_FG_i, ff_avg_FG_i, poly_splits_ds = ctc.split_and_build_FF_FG(input_FG,n_splits, ds_dim=12, us_dim=img_fg.shape[1])

In [ ]:
# REVIEW MINI-BATCH FG PROFILES (REMOVE ANY with CLEAR ARTIFACTS)
input_bg_sub = ff_avg_FG_i
n = input_bg_sub.shape[0]
for i in range(n):
    plt.figure(figsize=(1,1))  # Create a new figure for each slice
    plt.imshow(input_bg_sub[i], cmap='jet')  # Display the slice as an image
    plt.title(f'Slice {i+1}')  # Add a title for each plot
    plt.colorbar()  # Add a colorbar to show the intensity values
    plt.show()  # Show the plot

In [ ]:
# SAVE RESULTING FF HERE
bg_sub_avg = []
n = input_bg_sub.shape[0]
for i in range(n):
    print(i)
    bg_sub_i = input_bg_sub[i,...]
    bg_sub_i = bg_sub_i-ff_mask_BG_i
    bg_sub_avg.append(bg_sub_i)

bg_sub_avg = np.stack(bg_sub_avg, axis=0)
bg_sub_avg = np.nanmean(bg_sub_avg, axis=0)
ff_mask_FG_c = ctc.build_correction_mask(bg_sub_avg, 3)

FF_BG_norm = ff_mask_BG_i/np.nanmean(ff_mask_BG_i, axis=(0,1))
FF_BG_norm = ctc.eliminate_zeros_FF(FF_BG_norm)
plt.imshow(FF_BG_norm)
plt.colorbar()
plt.show()

FF_FG_norm = ff_mask_FG_c/np.nanmean(ff_mask_FG_c, axis=(0,1))
FF_FG_norm = ctc.eliminate_zeros_FF(FF_FG_norm)
plt.imshow(FF_FG_norm)
plt.colorbar()
plt.show()
